In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import UpSampling1D,Input, Concatenate, LSTM, Bidirectional, Dense, BatchNormalization, Dropout,  Layer, Attention, GlobalAveragePooling1D, Flatten,Conv1D, MaxPooling1D, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
from keras.regularizers import l2

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import time

In [ ]:
file_path = "/kaggle/input/balanced/RainInVn.csv"
df = pd.read_csv(file_path)
# df = df.drop(columns=['cloud'])
df = df.iloc[:, 1:]

In [ ]:
df.head(5)

In [ ]:
X = df.drop('RainTomorrow', axis=1).values
Y = df['RainTomorrow'].values
Y = Y.reshape(-1,1)

In [ ]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
import math
from keras.layers import *
from keras.layers import Activation
from keras.layers import GlobalAveragePooling2D
import keras.backend as K
import tensorflow as tf

def eca_layer(inputs_tensor=None, num=None, gamma=2, b=1, **kwargs):
    """
    ECA-NET
    :param inputs_tensor: input_tensor.shape=[batchsize,h,w,channels]
    :param num:
    :param gamma:
    :param b:
    :return:
    """
    channels = K.int_shape(inputs_tensor)[-1]
    t = int(abs((math.log(channels, 2) + b) / gamma))
    k = t if t % 2 else t + 1
    
    x = Conv1D(1, kernel_size=k, padding="same", name="eca_conv1_" + str(num))(inputs_tensor)
    x = Activation('sigmoid', name='eca_conv1_relu_' + str(num))(x)  #shape=[batch,chnnels,1]
    output = multiply([inputs_tensor, x])
    return output

In [ ]:
input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))

x1 = Bidirectional(LSTM(256, return_sequences=True))(input_layer)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)

x1 = Bidirectional(LSTM(128, return_sequences=True))(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.5)(x1)

x1 = eca_layer(inputs_tensor=x1)

query_value_attention_seq = Attention()([x1, x1])
query_value_attention = GlobalAveragePooling1D()(query_value_attention_seq)

output1 = Dense(1, activation='sigmoid')(x1)

model1 = Model(input_layer, output1)

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model1.summary()

In [ ]:
cp = ModelCheckpoint('model1/', save_best_only=True)
stop_early = EarlyStopping(monitor='val_loss', patience=5)
history = model1.fit(X_train, Y_train, batch_size=64, epochs=30, validation_split=0.2, callbacks=[cp,stop_early])

In [ ]:
model1 = load_model('/kaggle/working/model1')

In [ ]:
s = time.time()
predictions = model1.predict(X_test)
e = time.time()
predictions = np.where(predictions > 0.5, 1, 0)

predictions = predictions[:, -1, 0]

precision = precision_score(Y_test, predictions)
recall = recall_score(Y_test, predictions)
accuracy = accuracy_score(Y_test, predictions)
f1 = f1_score(Y_test, predictions)

In [ ]:
print(f"Precision: {precision:.5f}")
print(f"Recall: {recall:.5f}")
print(f"Accuracy: {accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")
print(f"Time: {(e-s):.5f}")

In [ ]:
model1.save(f"VN_ECA_Attention_{accuracy:.5f}.h5")